In [ ]:
import polars as pl
import networkx as nx
import time
import rtsvg
from linknode_graph_patterns import LinkNodeGraphPatterns
rt = rtsvg.RACETrack()
def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    g      = _patterns_.createPattern(_type_)
    _lms_  = rt.identifyLandmarks(g)
    _node_colors_ = {}
    for _lm_ in _lms_[0]: _node_colors_[_lm_] = 'red'
    df     = graphAsDataFrame(g)
    params = {'df':df, 'relationships':[('fm','to')], 'w':256, 'h':256, 'node_size':'small', 'link_arrow':False, 'node_color':_node_colors_}

    t0 = time.time()
    pos_nx = nx.spring_layout(g)
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_nx,  **params), f'{_type_} Spring (NX) {t1-t0:.2f}s'))

    t0 = time.time()
    pos_ref = rt.springLayout(g)
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_ref, **params), f'Spring (RT) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pfdl_k0    = rtsvg.PolarsForceDirectedLayout(g, k=0)
    pos_psl_k0 = pfdl_k0.results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl_k0, **params), f'Polar_K_0 (RT) {t1-t0:.2f}s'))
                   
    t0 = time.time()
    pfdl_k1    = rtsvg.PolarsForceDirectedLayout(g, k=1)
    pos_psl_k1 = pfdl_k1.results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl_k1, **params), f'Polar_K_1 (RT) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pfdl_k2    = rtsvg.PolarsForceDirectedLayout(g, k=2)
    pos_psl_k2 = pfdl_k2.results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl_k2, **params), f'Polar_K_2 (RT) {t1-t0:.2f}s'))

    break
rt.table(_tiles_, per_row=5, spacer=10)

In [ ]:
#
# Attempting to match Figure 1 from "Drawing Graphs To Convey Proximity"
# - looks like i don't do enough iterations...
#
from math import log10

_tiles_ = []
g       = _patterns_.createPattern('X')
df      = graphAsDataFrame(g)
for k in range(3):
    _lu_ = {'iteration':[], 'stress':[], 'trial':[], 'time':[]}
    for _trial_ in range(25):
        t0    = time.time()
        pfdl  = rtsvg.PolarsForceDirectedLayout(g, k=k)
        t1    = time.time()
        _vec_ = pfdl.stressVector()
        for i in range(len(_vec_)):
            _lu_['iteration'].append(i), _lu_['stress'].append(log10(_vec_[i])), _lu_['trial'].append(_trial_), _lu_['time'].append(t1-t0)
    df_stress = pl.DataFrame(_lu_)
    _tiles_.append(rt.addTitleToSVG(rt.xy(df_stress, x_field='iteration', y_field='stress', line_groupby_field='trial', dot_size=None), f'k={k}'))
rt.tile(_tiles_, spacer=10)